# <center> 【Kaggle】Telco Customer Churn 电信用户流失预测案例

&emsp;&emsp;<font face="仿宋">在案例的第二部分中，我们详细介绍了常用特征转化方法，其中有些是模型训练之必须，如自然数编码、独热编码，而有些方法则是以提高数据质量为核心、在大多数时候都是作为模型优化的备选方法，如连续变量分箱、数据标准化等。当然，在此之后，我们首先尝试构建一些可解释性较强的模型来进行用户流失预测，即采用逻辑回归和决策树模型来进行预测，并同时详细介绍了两种模型在实战中的调优技巧，在最终模型训练完成后，我们也重点讨论了关于两种可解释性模型建模结果的解释方法。

&emsp;&emsp;<font face="仿宋">从理论上来说，树模型的判别能力是要强于逻辑回归的，但在上一节最后的建模结果中我们发现两个模型的建模并无显著差别，预测准确率都维持在79%-80%之间，这或许说明很多逻辑回归无法正确判别的样本决策树模型也无法判别，据此我们推测，这是一个“入门容易、精通较难”的数据集。当然，如果我们进一步尝试其他“更强”的集成学习算法，如随机森林、XGB、CatBoost等，在当前数据集上的建模结果和逻辑回归也并无太大差异，因此我们亟需通过特征工程方法进一步提升数据集质量，进而提升最终模型效果。

&emsp;&emsp;<font face="仿宋">当然，哪怕是复杂模型在当前数据集上表现出了更好的效果，采用特征工程方法提升数据质量仍是优化建模结果必不可少的部分，正如时下流行的描述那样，“数据质量决定模型上界，而建模过程只是不断逼近这个上界”，特征工程中的一系列提高数据质量的方法、无论是在工业界实践中还是各大顶级竞赛里，都已然成了最为重要的提升模型效果的手段。

<center><img src="https://tva1.sinaimg.cn/large/008i3skNly1gwllgk4wgqj31hr0u0wh4.jpg" alt="image-20211112170651500" style="zoom:15%;" />

&emsp;&emsp;<font face="仿宋">不过，所谓的通过特征工程方法提高数据质量，看似简单但实际操作起来却并不容易。其难点并不在于其中具体操作方法的理解，至少相比机器学习算法原理，特征工程的很多方法并不复杂，特征工程的最大难点在于配合模型与数据进行方法选择、以及各种方法的工程化部署实现。一方面，特征工程方法众多，需要根据实际情况“因地制宜”，但数据的情况千变万化，很多时候需要同时结合数据探索结论、建模人员自身经验以及对各种备选方法的熟悉程度，才能快速制定行之有效的特征工程策略；另一方面，很多特征工程方法不像机器学习算法有现成的库可以直接调用，很多方法、尤其是一些围绕当前数据集的定制方法，需要自己手动实现，而这个过程就对建模人员本身的代码编写能力及工程部署能力提出了更高的要求。总而言之，特征工程是一个实践高度相关的技术，这也是为何课程会在介绍案例的过程中同步介绍特征工程常用方法的原因。

&emsp;&emsp;<font face="仿宋">当然，从宽泛的角度来看，所有围绕数据集的数据调整工作都可以看成是特征工程的一部分，包括此前介绍的缺失值填补、数据编码、特征变换等，这些方法其实都能一定程度提升数据质量，而本节开始，我们将花费一整节的时间来讨论另一类特征工程方法：特征衍生与特征筛选。而该方法通过创建更多特征来提供更多捕捉数据规律的维度，从而提升模型效果。当然特征衍生也是目前公认的最为有效的、能够显著提升数据集质量方法。

# <center>Part 2.交叉组合特征衍生策略

&emsp;&emsp;本阶开始我们将重点讨论特征工程中的特征衍生与特征筛选方法，并借此进一步提升模型效果。首先需要将此前的操作中涉及到的第三方库进行统一的导入：

In [1]:
# 基础数据科学运算库
import numpy as np
import pandas as pd

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# 时间模块
import time

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer

# 实用函数
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin

# 自定义模块
from telcoFunc import *

# re模块相关
import inspect, re

其中telcoFunc是自定义的模块，其内保存了此前自定义的函数和类，后续新增的函数和类也将逐步写入其中，telcoFunc.py文件随课件提供，需要将其放置于当前ipy文件同一文件夹内才能正常导入。

&emsp;&emsp;接下来导入数据并执行Part 1中的数据清洗步骤。

In [2]:
# 读取数据
tcc = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 标注连续/离散字段
# 离散字段
category_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
                'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
                'PaymentMethod']

# 连续字段
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
 
# 标签
target = 'Churn'

# ID列
ID_col = 'customerID'

# 验证是否划分能完全
assert len(category_cols) + len(numeric_cols) + 2 == tcc.shape[1]

# 连续字段转化
tcc['TotalCharges']= tcc['TotalCharges'].apply(lambda x: x if x!= ' ' else np.nan).astype(float)
tcc['MonthlyCharges'] = tcc['MonthlyCharges'].astype(float)

# 缺失值填补
tcc['TotalCharges'] = tcc['TotalCharges'].fillna(0)

# 标签值手动转化 
tcc['Churn'].replace(to_replace='Yes', value=1, inplace=True)
tcc['Churn'].replace(to_replace='No',  value=0, inplace=True)

In [3]:
features = tcc.drop(columns=[ID_col, target]).copy()
labels = tcc['Churn'].copy()

接下来即可直接带入数据进行特征衍生。

### 2.交叉组合特征衍生

- 方法介绍

&emsp;&emsp;所谓交叉组合特征衍生，指的是不同分类变量不同取值水平之间进行交叉组合，从而创建新字段的过程。例如此前我们创建的老年且经济不独立的标识字段，实际上就是是否是老年人字段（SeniorCitizen）和是否经济独立字段（Dependents）两个字段交叉组合衍生过程中的一个：

<center><img src="https://s2.loli.net/2022/01/20/BiH4LtVTOjkWQuI.png" alt="image-20220120140301256" style="zoom:33%;" />

不难看出，该计算流程并不复杂，需要注意的是，交叉组合后衍生的特征个数是参数交叉组合的特征的取值水平之积，因此交叉组合特征衍生一般只适用于取值水平较少的分类变量之间进行，若是分类变量或者取值水平较多的离散变量彼此之间进行交叉组合，则会导致衍生特征矩阵过于稀疏，从而无法为模型提供有效的训练信息。

- 手动实现

&emsp;&emsp;我们仍然以telco数据集为例，尝试围绕'SeniorCitizen'、'Partner'、'Dependents'字段进行两两交叉组合衍生，当然该流程也可以顺利推广至任意多个任意取值个数的分类变量两两交叉组合衍生过程。

In [7]:
# 数据集中离散变量
category_cols

['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod']

In [8]:
# 提取目标字段
colNames = ['SeniorCitizen', 'Partner', 'Dependents']

In [9]:
# 单独提取目标字段的数据集
features_temp = features[colNames]
features_temp.head(5)

,SeniorCitizen,Partner,Dependents
0,0,Yes,No
1,0,No,No
2,0,No,No
3,0,No,No
4,0,No,No


In [10]:
# 创建空列表用于存储衍生后的特征名称和特征
colNames_new_l = []
features_new_l = []

In [11]:
# enumerate过程
for col_index, col_name in enumerate(colNames):
    print(col_index, col_name)

0 SeniorCitizen
1 Partner
2 Dependents


In [12]:
# 衍生特征列名称
for col_index, col_name in enumerate(colNames):
    for col_sub_index in range(col_index+1, len(colNames)):
        newNames = col_name + '&' + colNames[col_sub_index]
        print(newNames)

SeniorCitizen&Partner
SeniorCitizen&Dependents
Partner&Dependents


In [13]:
# 创建衍生特征列名称及特征本身
for col_index, col_name in enumerate(colNames):
    for col_sub_index in range(col_index+1, len(colNames)):
        
        newNames = col_name + '&' + colNames[col_sub_index]
        colNames_new_l.append(newNames)
        
        newDF = pd.Series(features[col_name].astype('str') 
                          + '&'
                          + features[colNames[col_sub_index]].astype('str'), 
                          name=col_name)
        features_new_l.append(newDF)

In [14]:
features_new = pd.concat(features_new_l, axis=1)
features_new.columns = colNames_new_l

In [15]:
features_new

,SeniorCitizen&Partner,SeniorCitizen&Dependents,Partner&Dependents
0,0&Yes,0&No,Yes&No
1,0&No,0&No,No&No
2,0&No,0&No,No&No
3,0&No,0&No,No&No
4,0&No,0&No,No&No
...,...,...,...
7038,0&Yes,0&Yes,Yes&Yes
7039,0&Yes,0&Yes,Yes&Yes
7040,0&Yes,0&Yes,Yes&Yes
7041,1&Yes,1&No,Yes&No


In [16]:
colNames_new_l

['SeniorCitizen&Partner', 'SeniorCitizen&Dependents', 'Partner&Dependents']

截至目前，我们创建了3个4分类的变量，我们可以直接将其带入进行建模，但需要知道的是这些四分类变量并不是有序变量，因此往往我们需要进一步将这些衍生的变量进行独热编码，然后再带入模型：

In [17]:
enc = preprocessing.OneHotEncoder()

In [18]:
enc.fit_transform(features_new)

<7043x12 sparse matrix of type '<class 'numpy.float64'>'
	with 21129 stored elements in Compressed Sparse Row format>

In [19]:
# 借助此前定义的列名称提取器进行列名称提取
cate_colName(enc, colNames_new_l, drop=None)

['SeniorCitizen&Partner_0&No',
 'SeniorCitizen&Partner_0&Yes',
 'SeniorCitizen&Partner_1&No',
 'SeniorCitizen&Partner_1&Yes',
 'SeniorCitizen&Dependents_0&No',
 'SeniorCitizen&Dependents_0&Yes',
 'SeniorCitizen&Dependents_1&No',
 'SeniorCitizen&Dependents_1&Yes',
 'Partner&Dependents_No&No',
 'Partner&Dependents_No&Yes',
 'Partner&Dependents_Yes&No',
 'Partner&Dependents_Yes&Yes']

In [20]:
# 最后创建一个完整的衍生后的特征矩阵
features_new_af = pd.DataFrame(enc.fit_transform(features_new).toarray(), 
                               columns = cate_colName(enc, colNames_new_l, drop=None))

In [21]:
features_new_af.head(5)

,SeniorCitizen&Partner_0&No,SeniorCitizen&Partner_0&Yes,SeniorCitizen&Partner_1&No,SeniorCitizen&Partner_1&Yes,SeniorCitizen&Dependents_0&No,SeniorCitizen&Dependents_0&Yes,SeniorCitizen&Dependents_1&No,SeniorCitizen&Dependents_1&Yes,Partner&Dependents_No&No,Partner&Dependents_No&Yes,Partner&Dependents_Yes&No,Partner&Dependents_Yes&Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


至此，我们就完整的完成了既定变量的两两交叉衍生过程，我们可以将上述过程封装为如下函数：

- 函数封装

In [12]:
def Binary_Cross_Combination(colNames, features, OneHot=True):
    """
    分类变量两两组合交叉衍生函数
    
    :param colNames: 参与交叉衍生的列名称
    :param features: 原始数据集
    :param OneHot: 是否进行独热编码
    
    :return：交叉衍生后的新特征和新列名称
    """
    
    # 创建空列表存储器
    colNames_new_l = []
    features_new_l = []
    
    # 提取需要进行交叉组合的特征
    features = features[colNames]
    
    # 逐个创造新特征名称、新特征
    for col_index, col_name in enumerate(colNames):
        for col_sub_index in range(col_index+1, len(colNames)):
            
            newNames = col_name + '&' + colNames[col_sub_index]
            colNames_new_l.append(newNames)
            
            newDF = pd.Series(features[col_name].astype('str')  
                              + '&'
                              + features[colNames[col_sub_index]].astype('str'), 
                              name=col_name)
            features_new_l.append(newDF)
    
    # 拼接新特征矩阵
    features_new = pd.concat(features_new_l, axis=1)
    features_new.columns = colNames_new_l
    colNames_new = colNames_new_l
    
    # 对新特征矩阵进行独热编码
    if OneHot == True:
        enc = preprocessing.OneHotEncoder()
        enc.fit_transform(features_new)
        colNames_new = cate_colName(enc, colNames_new_l, drop=None)
        features_new = pd.DataFrame(enc.fit_transform(features_new).toarray(), columns=colNames_new)
        
    return features_new, colNames_new

> 这里需要注意，本节定义的特征衍生函数都将创建衍生列的特征名称，同时输出的数据也是衍生后的新的特征矩阵，而非和原数据拼接后的结果，这也将为后续使用多种方法、创建多个衍生特征矩阵、再进行统一拼接提供便捷。

简单验证上述函数执行过程：

In [23]:
features_new, colNames_new = Binary_Cross_Combination(colNames, features)

In [24]:
features_new.head(5)

,SeniorCitizen&Partner_0&No,SeniorCitizen&Partner_0&Yes,SeniorCitizen&Partner_1&No,SeniorCitizen&Partner_1&Yes,SeniorCitizen&Dependents_0&No,SeniorCitizen&Dependents_0&Yes,SeniorCitizen&Dependents_1&No,SeniorCitizen&Dependents_1&Yes,Partner&Dependents_No&No,Partner&Dependents_No&Yes,Partner&Dependents_Yes&No,Partner&Dependents_Yes&Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [25]:
colNames_new

['SeniorCitizen&Partner_0&No',
 'SeniorCitizen&Partner_0&Yes',
 'SeniorCitizen&Partner_1&No',
 'SeniorCitizen&Partner_1&Yes',
 'SeniorCitizen&Dependents_0&No',
 'SeniorCitizen&Dependents_0&Yes',
 'SeniorCitizen&Dependents_1&No',
 'SeniorCitizen&Dependents_1&Yes',
 'Partner&Dependents_No&No',
 'Partner&Dependents_No&Yes',
 'Partner&Dependents_Yes&No',
 'Partner&Dependents_Yes&Yes']

当然，完成衍生特征矩阵创建后，还需要和原始数据集进行拼接，此处拼接过程较为简单，直接使用concat函数即可：

In [26]:
df_temp = pd.concat([features, features_new], axis=1)
df_temp.head(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,SeniorCitizen&Partner_1&No,SeniorCitizen&Partner_1&Yes,SeniorCitizen&Dependents_0&No,SeniorCitizen&Dependents_0&Yes,SeniorCitizen&Dependents_1&No,SeniorCitizen&Dependents_1&Yes,Partner&Dependents_No&No,Partner&Dependents_No&Yes,Partner&Dependents_Yes&No,Partner&Dependents_Yes&Yes
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


至此，我们就完成了双变量交叉组合衍生的全过程。这里我们不着急带入新的特征进入模型进行效果测试，对于大多数批量创建特征的方法来说，创建的海量特征往往无效特征占绝大多数，例如此前我们曾手动验证过老年无伴侣字段就是无效字段。因此，如果不配合特征筛选方法、盲目带入大量无用特征进入模型，不仅不会起到正向的提升效果，往往可能还会适得其反。待后续介绍特征筛选方法后，我们再来看这些衍生出来的特征效用如何。

- 使用时注意事项

&emsp;&emsp;在实际使用过程中，双变量的交叉衍生是最常见的特征衍生方法，也是第一梯队优先考虑的特征衍生的策略。通过不同分类水平的交叉衍生，能够极大程度丰富数据集信息呈现形式，同时也为有效信息的精细化筛选提供了更多可能。

&emsp;&emsp;但同时也需要注意，越多的分类特征进行交叉组合、或者参与交叉组合的特征本身分类水平更多，衍生的特征数量也将指数级上涨，例如有10个二分类变量参与交叉衍生，则最终将衍生出$2^{10}=1024$个新特征，而如果是10个三分类变量参与交叉衍生，则最终将衍生出$3^{10}=29049$个新特征。无论如何进行衍生，首先我们需要对衍生后的特征规模有基本判断。

### 1.多变量的交叉组合特征衍生

- 原理介绍

&emsp;&emsp;接下来进一步讨论多变量的交叉组合特征衍生。同样，所谓多变量的交叉组合，就是将多个特征的不同取值水平进行组合，基本过程如下所示：

<center><img src="https://s2.loli.net/2022/02/11/AjeRWLpfOQTwGHF.png" alt="image-20220211160030921" style="zoom:33%;" />

该过程并不复杂，但需要注意的是，伴随着交叉组合特征数量的增加、以及每个特征取值水平增加，衍生出来的特征数量将呈指数级上涨趋势，例如3个包含两个分类水平的离散变量进行交叉组合时，将衍生出$2^3=8$个特征，而如果是10个包含三个分类水平的离散变量进行交叉组合，则将衍生出$3^{10}=59049$个特征。当然，特征数量的多并没有太大影响，但如果同时特征矩阵过于稀疏（有较多零值），则表示相同规模数据情况下包含了较少信息，而这也将极大程度影响后续建模过程。

&emsp;&emsp;通过上述极简示例不难看出，通过交叉组合衍生出来的新特征矩阵是极度稀疏的（即0值占绝大多数），并且不难发现，每一行在衍生的特征矩阵中其实只有一个值是1，其余值都是0（只有组合出了一种取值）。可以证明，在m个n分类特征的交叉组合过程中，假设总共有k条数据，则0值的占比为：

$$\frac{n^m*k-k}{n^m*k}=1-\frac{1}{n^m}$$

即如果是3个2分类水平的特征进行交叉组合衍生，则新的特征矩阵中0值占比为$1-\frac{1}{8}=\frac{7}{8}=87.5$%；而如果是10个三分类变量进行交叉组合衍生，则新特征矩阵中0值占比为$1-\frac{1}{3^{10}}=99.99831$%。尽管后续我们将介绍在海量特征中筛选有效特征的方法，但是在如此稀疏的矩阵中提取信息仍然还是一件非常困难的事情，因此往往我们并不会带入过多的特征进行交叉组合特征衍生。一般来说，如果有多个特征要进行交叉组合衍生，我们往往优先考虑两两组合进行交叉组合衍生，只有在人工判断是极为重要的特征情况下，才会考虑对其进行三个甚至更多的特征进行交叉组合衍生。